In [2]:
# Test Cell
# When the notebook is executed by the widget, this cell is ignored. 
# See the `Test Cell` section below for further details.
import pandas as pd
import SisenseAPI
import sys
import os
import base64
import io
import csv
import json

df_input = pd.DataFrame()

# Enter the required operation [Install / UnInstall] and your AIAPIKEY (Ex: openAI Key)
additional_parameters = '''{"operation":"Install","AIAPIKEY":"<ENTER_YOUR_KEY>"}'''
sisense_conn = SisenseAPI.SisenseAPI(sisense_user_authentication_token="XXX", schema_id="", query_building_cube=False, cube_name="XXX", additional_parameters="{}", is_token_cookie=True)

In [3]:
#Workaround for release 2023.2
sisense_conn.set_xsrf_token_header(sisense_conn.headers['Cookie'])

In [4]:
import os
import json
import re
import requests
import urllib.parse

add_param = json.loads(additional_parameters)
operation = add_param['operation']
AIAPIKEY = add_param['AIAPIKEY']
headers = {
    'Cookie': sisense_conn.headers['Cookie'],
    'X-XSRF-TOKEN': sisense_conn.headers['X-XSRF-TOKEN'],
}
sisense_base_url = 'http://' + os.environ['API_GATEWAY_EXTERNAL_SERVICE_HOST'] + ':' + os.environ['API_GATEWAY_EXTERNAL_SERVICE_PORT']


# Download ai-integration-examples files from GitHub   
    

In [ ]:
if operation == 'Install':
    !rm -rf ./ai-integration-examples
    !wget https://github.com/sisense/ai-integration-examples/archive/master.tar.gz -O ./ai-integration-examples.tgz
    !tar fxz ./ai-integration-examples.tgz
    !mv ai-integration-examples-main ai-integration-examples    
if operation == 'UnInstall':
    !rm -rf ./ai-integration-examples ./ai-integration-examples.tgz
df_result = df_input

# Get CustomCode Notebooks that already exist

In [ ]:
res = sisense_conn.call_api('GET','/api/v1/notebooks?notebookType=CustomCodeTransformation')
current_trans = json.loads(res.text)
for i in current_trans:
    print (i['id'],i['uuid'],i['codePath'])
    print (i)

# Handle CustomCode notebooks

In [ ]:
destination_full_path = '/opt/sisense/storage/notebooks/custom_code_notebooks/notebooks/'
source_notebook_files_path = './ai-integration-examples/custom_code_notebooks'
if operation == 'Install':
    notebook_import_res = {}
    # import notebok into Sisense    
    for dir_name in os.listdir(source_notebook_files_path):
        path = os.path.join(source_notebook_files_path, dir_name)
        if not os.path.isdir(path):
            continue
            
        for fname in os.listdir(path):                   
            if ".sipynb" in fname:
                print(path +'/'+ fname)                
                with open(path +'/'+ fname) as f:
                    data = json.load(f)    
                    data['notebookType'] = "CustomCodeTransformation"
                    res = sisense_conn.call_api('POST','/api/v1/notebooks',payload=data)
                    print (res)
                    print (res.text)
                    notebook_import_res[fname] = json.loads(res.text)
                    
    # move notebook flow image to notebook dir                
    for dir_name in os.listdir(source_notebook_files_path):
        path = os.path.join(source_notebook_files_path, dir_name)
        if not os.path.isdir(path):
            continue
            
        for fname in os.listdir(path):
            if ".png" in fname:
                source_image_path = path +'/'+ fname
                destination_image_path = destination_full_path+dir_name
                os.system(f"cp -R {source_image_path} {destination_image_path}")
                print (f'copying notebook flow image: {source_image_path}->{destination_image_path}')                                        
            
# we separete the above two steps - since we don't want to override the dircetories.  

current_notebooks_name = [
    'Data Search',
    'Dashboard Summary',
    'Create Relations',
    'WidgetSummary',
    'SuggestRelation'
]
if operation == 'UnInstall':
    for i in current_trans:
        if i['notebookType'] == 'CustomCodeTransformation':
            if i['displayName'] in current_notebooks_name:
                url_suffix = '/api/v1/notebooks/'+i['uuid']
                url = sisense_base_url + url_suffix
                res = requests.delete( url,  headers= sisense_conn.headers , timeout=350)
                print(res)              
                path_to_remove = destination_full_path+i['id']
                os.system(f'rm -rf {path_to_remove}')               
                print ('deleteing: ', i['displayName'],i['uuid'], path_to_remove)       

# Handle Blox files

In [ ]:
## Push blox actions
blox_files_path = "./ai-integration-examples/blox_examples/action_snippets"

bloxToNotebook = {
    "bloxDataSearch": "DataSearch.sipynb",
    "bloxDataConnect": "CreateRelations.sipynb",
    "bloxWidgetSummary": "WidgetSummary.sipynb",
    "bloxDataSuggest": "SuggestRelation.sipynb"
}
bloxType = {
    "DataSuggest",
    "DataSearch",
    "DataConnect",
    "WidgetSummary",
    "DashboardSummary"
}
if operation == 'Install':
    for fname in os.listdir(blox_files_path):
        if ".json" in fname and "blox" in fname:
            print (fname)
            with open(blox_files_path +'/'+ fname) as f:
                data = json.load(f)
                res = sisense_conn.call_api('POST','/api/v1/saveCustomAction/BloX',payload=data)
                print (res)
if operation == 'UnInstall':
    res = sisense_conn.call_api('GET','/api/v1/getSnippets/BloX?snippetType=actions')
    res_json = json.loads(res.text)
    for i in res_json:
        if i['title'] == 'My Actions':
            for element in i['elements']:
                print (element)
                if element['title'] in bloxType:
                    data = { "type": element['title'] }
                    res = sisense_conn.call_api('POST','/api/v1/deleteCustomAction/BloX',payload=data)
                    print (element['title'])
                    print (res)

# Handle dashboard

In [ ]:
dashboard_file_path = './ai-integration-examples/common/BloxAI.dash'
if operation == 'Install':
    with open(dashboard_file_path) as f:
        data = json.load(f)    
        res = sisense_conn.call_api('POST','/api/v1/dashboards/import/bulk',payload=[data])
        print(res.text)
if operation == 'UnInstall':
    res = sisense_conn.call_api('GET','/api/v1/dashboards')
    res_json = json.loads(res.text)
    for i in res_json:
        if i['title'] == 'Blox AI':
            url_suffix = '/api/v1/dashboards/'+i['oid']
            url = sisense_base_url + url_suffix
            res = requests.delete( url,  headers= sisense_conn.headers , timeout=350)
            print (res.text)

# Handle custom python libraries
First, we will uncomment OpenAI API import in AIQueries.py file.
Later we will move the files to their relevant location.

In [ ]:
ai_queries_file_name = './ai-integration-examples/custom_code_notebooks/utils/AIQueries.py'
with open(ai_queries_file_name, 'r+') as file: 
    data = file.read()
    res = data.replace('# Un comment OPENAI #','')
    file.seek(0) 
    file.write(res) 
    file.truncate() 

In [ ]:
custom_libraries = ['AIIntegration.py', 'AIQueries.py', 'AIUtils.py', 'InferenceQuestionType.py']
destination_full_path = '/opt/sisense/storage/notebooks/custom_code_notebooks/'
source_path = "./ai-integration-examples/custom_code_notebooks/utils/" 

for custom_library in custom_libraries:
    custom_library_destination_path = destination_full_path+custom_library
    custom_library_source_path = source_path+custom_library
    if operation == 'Install':
        os.system(f"cp -R {custom_library_source_path} {custom_library_destination_path}")
    if operation == 'UnInstall': 
        os.system(f'rm -rf {custom_library_destination_path}')

# Handle default data model - AITransformation
import empty cube for calling transformation

In [ ]:
data_model_path = './ai-integration-examples/common/AITransformation.sdata'
if operation == 'Install':

    url_suffix = '/api/v2/datamodel-imports/stream/full?newTitle=AITransformation'
    url = sisense_base_url + url_suffix
    headers = {
        'Cookie': sisense_conn.headers['Cookie'],
        'X-XSRF-TOKEN': sisense_conn.headers['X-XSRF-TOKEN'],
        'Content-Encoding': 'gzip'
    }
    files = {'fileToUpload': open(data_model_path, 'rb')}
    res = requests.post( url,  headers= headers , files = files , timeout=350)

    print (res)
    print (res.text)
if operation == 'UnInstall':
    res = sisense_conn.call_api('POST','/api/elasticubes/LocalHost/AITransformation/stop')
    print (res.text)
    res = sisense_conn.call_api('GET','/api/v2/datamodels/schema')
    res_json = json.loads(res.text)
    for i in res_json:
        if i['title'] == "AITransformation":
            print (i['oid'],i['title'])
            url_suffix = '/api/v2/datamodels/'+i['oid']
            url = sisense_base_url + url_suffix
            res = requests.delete( url,  headers= sisense_conn.headers , timeout=350)
            print(res.text)

# Handle Blox images

In [12]:
if operation == 'Install':
    !mkdir /opt/sisense/storage/branding/BloxAI
    !cp ./ai-integration-examples/blox_examples/images/*.png /opt/sisense/storage/branding/BloxAI/
if operation == 'UnInstall':
    !rm -rf /opt/sisense/storage/branding/BloxAI

mkdir: cannot create directory ‘/opt/sisense/storage/branding/BloxAI’: File exists



# Handle BloxAISummary plugin

In [13]:
if operation == 'Install':
    plugins_path = './ai-integration-examples/blox_examples/plugin/BloxAISummary/bloxAI_summary.zip'
    url_suffix = '/api/v1/plugins/import?overwrite=true'
    url = sisense_base_url + url_suffix
    headers = {
        'Cookie': sisense_conn.headers['Cookie'],
        'X-XSRF-TOKEN': sisense_conn.headers['X-XSRF-TOKEN'],
        'Content-Encoding': 'gzip'
    }
    files = {'plugins': open(plugins_path, 'rb')}
    res = requests.post( url,  headers= headers , files = files , timeout=350)
    print (res.text)
if operation == 'UnInstall':
    url_suffix = '/api/v1/plugins/BloxAISummary%2FBloxAISummary'
    url = sisense_base_url + url_suffix
    res = requests.delete( url,  headers= sisense_conn.headers , timeout=350)


# Handle AI-API-KEY
insert AI-API-KEY to environment configuration

In [ ]:
def set_config_value(setting_dict):  # {<option_name>: <value>}
    # zk.set_path_value doesn't work from scratch. Settings can be changed manually or via API
    for option_name, value in setting_dict.items():
        print('Set config {} value {}'.format(option_name, value))
        payload = {'{}.value'.format(option_name): value, '{}.setbyuser'.format(option_name): True}
        resp = sisense_conn.call_api(http_method='POST', payload=payload,
                                         url_suffix='/app/configuration/configurations/service/custom-code')
        return resp
settings = {
    "AI-API-KEY": AIAPIKEY
}
set_config_value(settings)


# Install required libraries 

In [ ]:
# Uncomment the code below in order to install the openai package
# try:
#     import openai
# except ImportError:
#     !pip install openai

# try:
#     import strenum
# except ImportError:
#     !pip install StrEnum

In [ ]:
# monitor the df_result datatype (Optional)
df_result.dtypes